## Librerias a importar

In [11]:
import pandas as pd
import deepl
import os

In [2]:
# Importar datos
ruta_archivo = "./clientes_banco.csv"
df = pd.read_csv(ruta_archivo)

In [3]:
print(df.head(5))

   ID_Cliente  Idade Grau_escolaridade   Estado_civil  Tamanho_familia  \
0     5008804     32   Ensino superior  União-estável                2   
1     5008805     32   Ensino superior  União-estável                2   
2     5008806     58      Ensino médio         Casado                2   
3     5008808     52      Ensino médio       Solteiro                1   
4     5008809     52      Ensino médio       Solteiro                1   

    Categoria_de_renda   Ocupacao  Anos_empregado  Rendimento_anual  \
0            Empregado      Outro              12          427500.0   
1            Empregado      Outro              12          427500.0   
2            Empregado  Segurança               3          112500.0   
3  Associado comercial     Vendas               8          270000.0   
4  Associado comercial     Vendas               8          270000.0   

   Tem_carro                   Moradia  
0          1       Apartamento alugado  
1          1       Apartamento alugado  
2    

In [4]:
print(df.columns)

Index(['ID_Cliente', 'Idade', 'Grau_escolaridade', 'Estado_civil',
       'Tamanho_familia', 'Categoria_de_renda', 'Ocupacao', 'Anos_empregado',
       'Rendimento_anual', 'Tem_carro', 'Moradia'],
      dtype='object')


In [5]:
for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col}: {dtype}")

ID_Cliente: int64
Idade: int64
Grau_escolaridade: object
Estado_civil: object
Tamanho_familia: int64
Categoria_de_renda: object
Ocupacao: object
Anos_empregado: int64
Rendimento_anual: float64
Tem_carro: int64
Moradia: object


In [6]:
df.shape

(438463, 11)

In [7]:
translations = {
    'ID_Cliente': 'Customer_ID',
    'Idade': 'Age',
    'Grau_escolaridade': 'Education_Level',
    'Estado_civil': 'Marital_Status',
    'Tamanho_familia': 'Family_Size',
    'Categoria_de_renda': 'Income_Category',
    'Ocupacao': 'Occupation',
    'Anos_empregado': 'Years_Employed',
    'Rendimento_anual': 'Annual_Income',
    'Tem_carro': 'Has_Car',
    'Moradia': 'Housing'
}

In [8]:
df.rename(columns=translations, inplace=True)

In [9]:
df.head(5)

,Customer_ID,Age,Education_Level,Marital_Status,Family_Size,Income_Category,Occupation,Years_Employed,Annual_Income,Has_Car,Housing
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio


In [12]:
# Obtener la clave de autenticación de la API de DeepL desde el archivo .env
auth_key = os.getenv("DEEPL_AUTH_KEY")
translator = deepl.Translator(auth_key)

In [13]:
# Seleccionar columnas de texto
text_cols = df.select_dtypes(include=['object', 'string']).columns

In [14]:
text_cols

Index(['Education_Level', 'Marital_Status', 'Income_Category', 'Occupation',
       'Housing'],
      dtype='object')

In [15]:
# Traducir valores únicos de cada columna y mostrar la cantidad de caracteres traducidos
for col in text_cols:
    # Imprime el nombre de la columna que se va a traducir
    print(f"Traduciendo columna: {col}")
    # Obtiene los valores únicos no nulos de la columna
    unique_vals = df[col].dropna().unique()
    # Suma la cantidad total de caracteres de los valores únicos
    total_chars = sum(len(str(val)) for val in unique_vals)
    # Traduce cada valor único usando la API de DeepL y almacena en un diccionario
    translations = {
        val: translator.translate_text(val, source_lang="PT", target_lang="EN-US").text
        for val in unique_vals
    }
    # Reemplaza los valores de la columna por sus traducciones
    df[col] = df[col].map(translations)
    # Imprime la cantidad de caracteres traducidos en la columna actual
    print(f"Caracteres traducidos en '{col}': {total_chars}")


Traduciendo columna: Education_Level
Caracteres traducidos en 'Education_Level': 84
Traduciendo columna: Marital_Status
Caracteres traducidos en 'Marital_Status': 42
Traduciendo columna: Income_Category
Caracteres traducidos en 'Income_Category': 64
Traduciendo columna: Occupation
Caracteres traducidos en 'Occupation': 194
Traduciendo columna: Housing
Caracteres traducidos en 'Housing': 122


In [16]:
df.head(5)

,Customer_ID,Age,Education_Level,Marital_Status,Family_Size,Income_Category,Occupation,Years_Employed,Annual_Income,Has_Car,Housing
0,5008804,32,Higher education,Union-stable,2,Employee,Other,12,427500.0,1,Rented apartment
1,5008805,32,Higher education,Union-stable,2,Employee,Other,12,427500.0,1,Rented apartment
2,5008806,58,High school,Married,2,Employee,Security,3,112500.0,1,Own house/apartment
3,5008808,52,High school,Single,1,Business associate,Sales,8,270000.0,0,Own house/apartment
4,5008809,52,High school,Single,1,Business associate,Sales,8,270000.0,0,Own house/apartment


In [17]:
df.to_csv("clientes_banco_traducido.csv", index=False)